In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import time
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [2]:
print(f"Torch version= {torch.__version__},\nCuda version={torch.version.cuda}")

Torch version= 1.12.1+cu116,
Cuda version=11.6


In [3]:
df = pd.read_csv("../data/data.csv")
df.head()

,weekday_name,month,leap_year_condition,decade,output,output_year_digit,output_year,output_day,valid_years_days_
0,2,1,0,180,1-1-1800,0,1800,1,"{0: [1, 8, 15, 22, 29], 1: [7, 14, 21, 28], 2:..."
1,3,1,0,180,1-1-1801,1,1801,1,"{0: [2, 9, 16, 23, 30], 1: [1, 8, 15, 22, 29],..."
2,4,1,0,180,1-1-1802,2,1802,1,"{0: [3, 10, 17, 24, 31], 1: [2, 9, 16, 23, 30]..."
3,5,1,0,180,1-1-1803,3,1803,1,"{0: [4, 11, 18, 25], 1: [3, 10, 17, 24, 31], 2..."
4,6,1,1,180,1-1-1804,4,1804,1,"{4: [1, 8, 15, 22, 29], 8: [3, 10, 17, 24, 31]}"


**A- training for getting the year**

In [4]:
# fix random number generation aka regenerate the same random numbers every time (such as weight and bias initialization )
def set_random_seed(seed=7, deterministic=True):
    """Set random seed, for python, numpy, pytorch

    Args:
        seed (int): Seed to be used.
        deterministic (bool): Whether to set the deterministic option for
            CUDNN backend, i.e., set `torch.backends.cudnn.deterministic`
            to True and `torch.backends.cudnn.benchmark` to False.
            Default: True.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False 
seed=7        
set_random_seed(seed=seed)

In [5]:
# output_year column is just the last number in the output column
x_year = df[["leap_year_condition", "decade", "valid_years_days_", "output_year_digit"]]
y_year = x_year.pop("output_year_digit")
v = x_year.pop("valid_years_days_")
x_year.head(7)

,leap_year_condition,decade
0,0,180
1,0,180
2,0,180
3,0,180
4,1,180
5,0,180
6,0,180


In [6]:
batch_size = 64
xtrain_year, xtest_year, ytrain_year, ytest_year, vtrain_year, vtest_year = train_test_split(x_year, y_year, v, test_size=0.19, shuffle=True, random_state=seed)


class CollectedData(Dataset):
    def __init__(self, x, y):
        self.data = torch.tensor(x.values.astype(np.float32))
        self.label = torch.tensor(y.values)
        self.n_smpl = x.shape[0]
        
    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]
    
    def __len__(self):
        return self.n_smpl

In [7]:
train_set_year = CollectedData(xtrain_year, ytrain_year)
test_set_year = CollectedData(xtest_year, ytest_year)

In [8]:
# # this isn't suitable here, so that i used train_test_split function
# # train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-test_len, int(len(dataset)*0.2)])

# train_labels_year = torch.tensor(ytrain_year.values.astype(np.float32)) 
# test_labels_year = torch.tensor(ytest_year.values.astype(np.float32)) 
# train_input_year = torch.tensor(xtrain_year.values.astype(np.float32)) 
# test_input_year = torch.tensor(xtest_year.values.astype(np.float32)) 

# train_set_year = TensorDataset(train_input_year, train_labels_year)
# test_set_year = TensorDataset(test_input_year, test_labels_year)


In [9]:
train_loader_year = DataLoader(dataset=train_set_year, shuffle=True, batch_size=batch_size)
test_loader_year = DataLoader(dataset=test_set_year, batch_size=batch_size) 

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class network(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.fc1 = nn.Linear(in_features, 11)
        self.fc2 = nn.Linear(11, 11)
        self.fc3 = nn.Linear(11, 11)
        self.fc4 = nn.Linear(11, out_features) 
        self.initialize_weights()
        
        
    def forward(self, x):
        out = F.leaky_relu(self.fc1(x))
        out = F.leaky_relu(self.fc2(out))
        out = F.leaky_relu(self.fc3(out))
        out = (self.fc4(out))#nn.Softmax
        return out
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)

In [11]:
# calculating accuracy
@torch.no_grad()
def calculate_accuracy(model, data_loader=train_loader_year):
    model.eval()
    
    num_correct = 0
    num_samples = 0

    for data, labels in data_loader:
        
        # transfering data to cuda
        data = data.to(device=device)
        labels = labels.to(device=device)
                
        preds = model(data)
        num_correct += (preds.argmax(dim=1) == labels).sum().item()
        num_samples += len(labels)
    accuracy = num_correct/num_samples
    model.train()
    return accuracy  

In [12]:
# initializing the network
model = network(in_features=2, out_features=10).to(device)
lr = 0.05
# loss and optimizer initializing
criterion = nn.CrossEntropyLoss()
# criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
schedular = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.001, patience=3, verbose=True)

In [ ]:
def train_model(data_loader=train_loader_year):
    num_batches = len(data_loader)
    num_epochs = 10
    # starting training loop epochs
    for epoch in range(num_epochs):
        print(f"starting learning rate = {lr} \n number of epochs = {num_epochs} \n number of batches = {num_batches} \n")
        progress = tqdm(enumerate(data_loader), total=num_batches, leave=True)
        for batch_idx, (data, labels) in progress: 

            # convert data to device
            data = data.to(device=device)
            labels = labels.to(device=device)        

            # getting prediction and loss
            preds = model(data)
            loss = criterion(preds, labels)        

            # back propagation
            optimizer.zero_grad()
            loss.backward()

            # optimization step
            optimizer.step()

            progress.set_description(f"epoch [{1+epoch}/{num_epochs}], loss={loss.item():0.4f}")
            progress.set_postfix()

        acc = calculate_accuracy(model, data_loader=train_loader_year)
        schedular.step(acc)
        print(f"after {1+epoch} epoch, accuracy = {(acc*100):.2f}%")
train_model(data_loader=train_loader_year)

starting learning rate = 0.05 
 number of epochs = 10 
 number of batches = 3708 



epoch [1/10], loss=2.3324:  20%|█████████▉                                         | 724/3708 [00:09<00:27, 107.88it/s]

In [ ]:
x_day = df[["weekday_name", "month", "output_year", "output_day"]]
x_day.head()

In [ ]:
y_day = x_day.pop("output_day")

In [ ]:
batch_size = 64
xtrain_day, xtest_day, ytrain_day, ytest_day, vtrain_day, vtest_day = train_test_split(x_day, y_day, v, test_size=0.19, shuffle=True, random_state=seed)

In [ ]:
train_set_day = CollectedData(xtrain_day, ytrain_day)
test_set_day = CollectedData(xtest_day, ytest_day)

In [ ]:
# # this isn't suitable here, so that i used train_test_split function
# # train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-test_len, int(len(dataset)*0.2)])

# train_labels_year = torch.tensor(ytrain_year.values.astype(np.float32)) 
# test_labels_year = torch.tensor(ytest_year.values.astype(np.float32)) 
# train_input_year = torch.tensor(xtrain_year.values.astype(np.float32)) 
# test_input_year = torch.tensor(xtest_year.values.astype(np.float32)) 

# train_set_year = TensorDataset(train_input_year, train_labels_year)
# test_set_year = TensorDataset(test_input_year, test_labels_year)


In [ ]:
train_loader_day = DataLoader(dataset=train_set_day, shuffle=True, batch_size=batch_size)
test_loader_day = DataLoader(dataset=test_set_day, batch_size=batch_size) 

In [ ]:
train_model(data_loader=test_loader_year)